## Image Classification

The basic 7 steps for building models in general are listed so:

1. Load Dataset
2. Make Dataset Iterable
3. Create Model Class
4. Instantiate Model Class
5. Instantiate Loss Class
6. Instantiate Optimizer Class
7. Train Model

In [108]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from torch.utils.data import Dataset, DataLoader

In [109]:
'''
STEP 1: LOAD DATASET
'''
# test_df = pd.read_csv('fashionmnist/fashion-mnist_test.csv')
# test_df_labels = test_df['label']
# test_pixels_df = test_df.drop('label', axis=1)
all_df = pd.read_table('list_eval_partition.txt', delim_whitespace=True)
validation_df = all_df[all_df['evaluation_status'].str.contains('val')]
# np.random.shuffle(validation_df)
validation_df = validation_df.drop(['evaluation_status'], axis=1)

train_df = all_df[all_df['evaluation_status'].str.contains('train')]
# np.random.shuffle(train_df)
train_df = train_df.drop(['evaluation_status'], axis=1)

test_df = all_df[all_df['evaluation_status'].str.contains('test')]
# np.random.shuffle(test_df)
test_df = test_df.drop(['evaluation_status'], axis=1)

labels_df = pd.read_table('list_category_img.txt', delim_whitespace=True)
labels_df

def func(image_name):
    category_label = labels_df[labels_df['image_name'].str.match(image_name)].iloc[0]['category_label']
    create_new = np.zeros(50)
    create_new[category_label - 1] = 1
    return create_new

# train_df = pd.read_csv('fashionmnist/fashion-mnist_train.csv')
# train_pixels_df = train_df.drop('label', axis=1)
# train_df_labels = train_df['label']


/home/paperspace/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  import sys
/home/paperspace/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


In [137]:
np.unique(np.argmax(labelvalidation, axis=1), return_counts=True)

(array([ 1,  2,  5,  8,  9, 10, 11, 14, 15, 16, 17, 18, 31]),
 array([ 4,  8,  5,  2,  1,  5,  1,  1,  3,  2, 13,  4,  1]))

In [136]:
np.unique(np.argmax(labeltrain, axis=1), return_counts=True)

(array([ 0,  1,  2,  4,  5,  6,  9, 10, 11, 14, 15, 16, 17, 18, 19]),
 array([ 1, 26, 97,  3, 22,  2,  7,  6,  1,  2, 15, 13, 46,  8,  1]))

In [157]:
allimagesvalidation = []
labelvalidation = []
for index in range(50):
#     create random number
    x = np.random.choice(20000)
    img_filepath = validation_df.iloc[x]['image_name']
    im = Image.open(img_filepath)
    labelvalidation.append(func(img_filepath))
    imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))
    imarr = (imarr - imarr.mean())/imarr.std()
    allimagesvalidation.append(imarr)

allimagestrain = []
labeltrain = []
for index in range(250):
    x = np.random.choice(50000)
    img_filepath = train_df.iloc[x]['image_name']
    im = Image.open(img_filepath)
    labeltrain.append(func(img_filepath))
    imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))
    imarr = (imarr - imarr.mean())/imarr.std()
    allimagestrain.append(imarr)

# allimagestest = []
# labeltest = []
# for index in range(10000):
#     img_filepath = test_df.iloc[index]['image_name']
#     im = Image.open(img_filepath)
#     labeltest.append(func(img_filepath))
#     imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))
#     #imarr = np.round((imarr - imarr.mean())/imarr.std())
#     allimagestest.append(imarr)
# im = Image.open('414m1dOolTL._SX342_.jpg')
# imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))


In [160]:
# '''
# STEP 1.5: defining and instantiating Dataset subclass 
# '''

# '''
# This is our custom Dataset class. Remember from 1st meeting that we need this to pipeline our data into training our model.

# The pipeline is important!!! At larger scale, machine learning can get bottlenecked at disk reads (in image classification for example)
# so understanding the various stages is important. We don't have to worry about that kind of stuff now since we're just creating small
# project models as opposed to complex production models.

# NOTE: this is not the only way to create a dataset. An alternative is to simply pass in a dataframe that contains both pixel and label data.
# Then we can index the label and pixel data inside of __getitem__ as opposed to separating labels and pixel data before hand like I did.
# '''
# class FashionDataset(Dataset):
#     def __init__(self, dataframe, labels):
#         self.labels = torch.LongTensor(labels)
#         self.df = dataframe
        
#     def __getitem__(self, index):
#         # I'm using .loc to access the row of the dataframe by index
#         a = self.df.loc[index]
# #         a = (a - np.mean(a))/np.std(a)
#         img = torch.Tensor(a)
#         label = self.labels[index]
#         return img, label

#     def __len__(self):
#         return len(self.labels)
    
# '''
# This class is for providing image data as (1, 28, 28) tensor as opposed to a (784) tensor. You
# use these for conv2d layers which are powerful for image recognition!

# NOTE: Please note that I normalized the data VERY INCORRECTLY. Here I am normalizing the data across 
# each sample individually which is not good. I should be normalizing across the ENTIRE training data set.

# Also, when I create the test dataset I should normalize it based on the TRAINING set's mean and standard deviation.
# Since the model is trained on the training data, we want to make sure that we transform the test data the same way we
# transform the training data. Otherwise it's like training a model to do one job and then testing it by on another job.
# '''
# class Fashion2DDataset(Dataset):
#     def __init__(self, dataframe, labels):
#         self.labels = torch.LongTensor(labels)
#         self.df = dataframe
        
#     def __getitem__(self, index):
#         # I'm using .loc to access the row of the dataframe by index
#         a = self.df.loc[index]
#         a = (a - np.mean(a))/np.std(a)
#         a = np.split(a, 28)
#         a = np.array([a])
#         img = torch.Tensor(a)
        
#         label = self.labels[index]
#         return img, label

#     def __len__(self):
#         return len(self.labels)

class ClothingDataset(Dataset):
    def __init__(self, data, labels):
        self.labels = labels
        self.data = data
        
    def __getitem__(self, index):
        img = self.data[index]
        
        trans = transforms.ToTensor()
        img = trans(img).float()
        
        label = self.labels[index]
        label = torch.LongTensor(label)
        
        return img, label

    def __len__(self):
        return len(self.labels)

In [161]:
'''
STEP 2: MAKING DATASET ITERABLE
'''
# train_dataset = Fashion2DDataset(train_pixels_df, train_df_labels.values)
# test_dataset = Fashion2DDataset(test_pixels_df, test_df_labels.values)

# batch_size = 100

# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
#                                            batch_size=batch_size, 
#                                            shuffle=True)

# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
#                                           batch_size=batch_size, 
#  shuffle=False)

batch_size = 25

train_dataset = ClothingDataset(allimagestrain, labeltrain)
validation_dataset = ClothingDataset(allimagesvalidation, labelvalidation) 
test_dataset = ClothingDataset(allimagestest, labeltest)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,                                             
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size,
                                          shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, 
                                          batch_size=batch_size,
                                          shuffle=True)




In [162]:
'''
STEP 3: CREATE MODEL CLASS
'''
# class FeedforwardNeuralNetModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(FeedforwardNeuralNetModel, self).__init__()
#         self.fc1 = nn.Linear(input_dim, hidden_dim) 
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_dim, output_dim)  

#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out
    
class ConvolutionalNeuralNetModel(nn.Module):
    def __init__(self, output_dim):
        super(ConvolutionalNeuralNetModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=56, stride=2, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(10)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.fc = nn.Linear(19360, output_dim)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [163]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
#input_dim = 224*224
#hidden_dim = 5000
output_dim = 50 

# model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)
model = ConvolutionalNeuralNetModel(output_dim)

In [164]:
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

In [165]:

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
"""
Most of the time I use SGD. Feel free to use another optimizer if you wish.
What hyperparameters would you use/set here?
"""
learning_rate = .1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [168]:
'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
losses = []
accuracies = []
for epoch in range(1):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
#         images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)
        labels = torch.max(labels,1)[1]
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 1 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in validation_loader:
                # Load images with gradient accumulation capabilities
                # images = images.view(-1, 28*28).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                labels = torch.max(labels,1)[1]

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy =  100 * correct / total
            
            accuracies.append(accuracy)
            losses.append(loss.item())
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 1. Loss: 3.6125376224517822. Accuracy: 14
Iteration: 2. Loss: 3.6951448917388916. Accuracy: 10
Iteration: 3. Loss: 3.6666314601898193. Accuracy: 20
Iteration: 4. Loss: 3.4955334663391113. Accuracy: 18
Iteration: 5. Loss: 3.431241035461426. Accuracy: 14
Iteration: 6. Loss: 3.2960643768310547. Accuracy: 18
Iteration: 7. Loss: 2.844761610031128. Accuracy: 16
Iteration: 8. Loss: 2.3488619327545166. Accuracy: 12
Iteration: 9. Loss: 2.1518523693084717. Accuracy: 22
Iteration: 10. Loss: 4.039227485656738. Accuracy: 14
